# Tutorial: The Basic Tools of Private Deep Learning -- TensorFlow edition!

Welcome to PySyft's introductory tutorial for privacy preserving, decentralized deep learning. This series of notebooks is a step-by-step guide for you to get to know the new tools and techniques required for doing deep learning on secret/private data/models without centralizing them under one authority.

**Scope:** Note that we'll not just be talking about how to decentralized / encrypt data, but we'll be addressing how PySyft can be used to help decentralize the entire ecosystem around data, even including the Databases where data is stored and queried, and the neural models which are used to extract information from data. As new extensions to PySyft are created, these notebooks will be extended with new tutorials to explain the new functionality.

Authors:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)

## Outline:

- Part 1: The Basic Tools of Private Deep Learning


## Why Take This Tutorial?

**1) A Competitive Career Advantage** - For the past 20 years, the digital revolution has made data more and more accessible in ever larger quantities as analog processes have become digitized. However, with new regulation such as [GDPR](https://eugdpr.org/), enterprises are under pressure to have less freedom with how they use - and more importantly how they analyze - personal information. **Bottom Line:** Data Scientists aren't going to have access to as much data with "old school" tools, but by learning the tools of Private Deep Learning, YOU can be ahead of this curve and have a competitive advantage in your career. 

**2) Entrepreneurial Opportunities** - There are a whole host of problems in society that Deep Learning can solve, but many of the most important haven't been explored because it would require access to incredibly sensitive information about people (consider using Deep Learning to help people with mental or relationship issues!). Thus, learning Private Deep Learning unlocks a whole host of new startup opportunities for you which were not previously available to others without these toolsets.

**3) Social Good** - Deep Learning can be used to solve a wide variety of problems in the real world, but Deep Learning on *personal information* is Deep Learning about people, *for people*. Learning how to do Deep Learning on data you don't own represents more than a career or entrepreneurial opportunity, it is the opportunity to help solve some of the most personal and important problems in people's lives - and to do it at scale.

## How do I get extra credit?

- Star PySyft on GitHub! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Star PySyft-TensorFlow on GitHub! - [https://github.com/OpenMined/PySyft-TensorFlow](https://github.com/OpenMined/PySyft-TensorFlow)
- Make a Youtube video teaching this notebook!


... ok ... let's do this!

# Part -1: Prerequisites

- Know TensorFlow - if not then take a look at the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/)
- Read the PySyft Framework Paper https://arxiv.org/abs/1811.04017! This will give you a thorough background on how PySyft is constructed, which will help things make more sense. Note the paper has gotten slightly out of date, so refer to these tutorials or ask in Slack if something seems strange! [slack.openmined.org](http://slack.openmined.org/)

# Part 0: Setup

To begin, you'll need to make sure you have the right things installed. To do so, head on over to PySyft's readme and follow the setup instructions. TLDR for most folks is.

- Install Python 3.6 or higher
- Install TensorFlow 2.0.0 (`pip install tensorflow==2.0.0`)
- TODO syft-tf install instructions

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then [open a GitHub issue](https://github.com/OpenMined/PySyft-TensorFlow/issues/new) or ping the #beginner channel in slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
# Run this cell to see if things work
import tensorflow as tf
import syft as sy
hook = sy.TensorFlowHook(tf)

tf.constant([1, 2, 3, 4, 5.])

<tf.Tensor: id=1, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>

If this cell executed, then you're off to the races! Let's do this!

# Part 1: The Basic Tools of Private, Decentralized Data Science

So - the first question you may be wondering is - How in the world do we train a model on data we don't have access to? 

Well, the answer is surprisingly simple. If you're used to working in TensorFlow, then you're used to working with tf.Tensor objects like these!

In [2]:
x = tf.constant([1, 2, 3, 4, 5.])
y = x + x
y

<tf.Tensor: id=3, shape=(5,), dtype=float32, numpy=array([ 2.,  4.,  6.,  8., 10.], dtype=float32)>

Obviously, using these super fancy (and powerful!) tensors is important, but also requires you to have the data on your local machine. This is where our journey begins. 

# Section 1.1 - Sending Tensors to Bob's Machine

Whereas normally we would perform data science / deep learning on the machine which holds the data, now we want to perform this kind of computation on some **other** machine. More specifically, we can no longer simply assume that the data is on our local machine.

Thus, instead of using TensorFlow tensors, we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [3]:
bob = sy.VirtualWorker(hook, id="bob")

Let's say Bob's machine is on another planet - perhaps on Mars! But, at the moment the machine is empty. Let's create some data so that we can send it to Bob and learn about pointers!

In [4]:
x = tf.constant([1., 2., 3., 4., 5.])
y = tf.constant([1., 1., 1., 1., 1.])

And now - let's send our tensors to Bob!!

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

BOOM! Now Bob has two tensors! Don't believe me? Have a look for yourself!

In [6]:
bob._objects

{31817784589: <tf.Tensor: id=8, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>,
 32714528385: <tf.Tensor: id=12, shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>}

In [7]:
z_ptr = x_ptr + x_ptr
z_ptr

(Wrapper)>[PointerTensor | me:31933462833 -> bob:8206513956]

In [8]:
bob._objects

{31817784589: <tf.Tensor: id=8, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>,
 32714528385: <tf.Tensor: id=12, shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>,
 8206513956: <tf.Tensor: id=14, shape=(5,), dtype=float32, numpy=array([ 2.,  4.,  6.,  8., 10.], dtype=float32)>}

Now notice something. When we called `x.send(bob)` it returned a new object that we called `x_ptr`. This is our first *pointer* to a tensor. Pointers to tensors do NOT actually hold data themselves. Instead, they simply contain metadata about a tensor (with data) stored on another machine. The purpose of these tensors is to give us an intuitive API to tell the other machine to compute functions using this tensor. Let's take a look at the metadata that pointers contain.

In [9]:
z_ptr

(Wrapper)>[PointerTensor | me:31933462833 -> bob:8206513956]

Check out that metadata!

There are two main attributes specific to pointers:

- `x_ptr.location : bob`, the location, a reference to the location that the pointer is pointing to
- `x_ptr.id_at_location : <random integer>`, the id where the tensor is stored at location

They are printed in the format `<id_at_location>@<location>`

There are also other more generic attributes:
- `x_ptr.id : <random integer>`, the id of our pointer tensor, it was allocated randomly
- `x_ptr.owner : "me"`, the worker which owns the pointer tensor, here it's the local worker, named "me"


In [10]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [11]:
bob

<VirtualWorker id:bob #objects:3>

In [12]:
bob == x_ptr.location

True

In [13]:
x_ptr.id_at_location

31817784589

In [14]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

You may wonder why the local worker which owns the pointer is also a VirtualWorker, although we didn't create it.
Fun fact, just like we had a VirtualWorker object for Bob, we (by default) always have one for us as well. This worker is automatically created when we construct `hook = sy.TensorFlowHook(tf)` and so you don't usually have to create it yourself.

In [15]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [16]:
me == x_ptr.owner

True

And finally, just like we can call .send() on a tensor, we can call .get() on a pointer to a tensor to get it back!!!

In [17]:
x_ptr.get()

<tf.Tensor: id=18, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>

In [18]:
y_ptr.get()

<tf.Tensor: id=21, shape=(5,), dtype=float32, numpy=array([1., 1., 1., 1., 1.], dtype=float32)>

In [19]:
z_ptr.get()

<tf.Tensor: id=24, shape=(5,), dtype=float32, numpy=array([ 2.,  4.,  6.,  8., 10.], dtype=float32)>

In [20]:
bob._objects

{}

And as you can see... Bob no longer has the tensors anymore!!! They've moved back to our machine!

# Section 1.2 - Using Tensor Pointers

So, sending and receiving tensors from Bob is great, but this is hardly Deep Learning! We want to be able to perform tensor _operations_ on remote tensors. Fortunately, tensor pointers make this quite easy! You can just use pointers like you would normal tensors!

In [21]:
x = tf.constant([1., 2., 3., 4., 5.]).send(bob)
y = tf.constant([1., 1., 1., 1., 1.]).send(bob)

In [22]:
z = x + y

In [23]:
z

(Wrapper)>[PointerTensor | me:35424001991 -> bob:87627708699]

And voilà! 

Behind the scenes, something very powerful happened. Instead of x and y computing an addition locally, a command was serialized and sent to Bob, who performed the computation, created a tensor z, and then returned the pointer to z back to us!

If we call .get() on the pointer, we will then receive the result back to our machine!

In [24]:
z.get()

<tf.Tensor: id=39, shape=(5,), dtype=float32, numpy=array([2., 3., 4., 5., 6.], dtype=float32)>

### TensorFlow Functions

This API has been extended to all of TensorFlow's primary operations!!!

In [25]:
x

(Wrapper)>[PointerTensor | me:26214408079 -> bob:35174604077]

In [26]:
y

(Wrapper)>[PointerTensor | me:53125201835 -> bob:40770790434]

In [27]:
z = tf.add(x, y)

In [28]:
z.get()

<tf.Tensor: id=44, shape=(5,), dtype=float32, numpy=array([2., 3., 4., 5., 6.], dtype=float32)>

### Variables [experimental]
Autodiff coming soon :)

In [29]:
x = tf.Variable([1., 2., 3., 4., 5.]).send(bob)
y = tf.Variable([1., 1., 1., 1., 1.]).send(bob)

In [30]:
coef = tf.constant(2.).send(bob)
z = x + coef * y

# manual differentiation
dzdy = coef
delta = tf.negative(dzdy)
delta = tf.broadcast_to(delta, [5])
y.assign_add(delta)

(Wrapper)>[PointerTensor | me:62848670961 -> bob:16896611751]

In [31]:
z = y.get()

In [32]:
print(z)

<tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([-1., -1., -1., -1., -1.], dtype=float32)>
